# Pruebas para la generacion de datos de entrenamiento

## Funciona sin utilizar clases

#### Importar librerias

In [1]:
import os
import math
import pickle
from zmqRemoteApi import RemoteAPIClient
from random import randint as rn

#### Inicializar la simulacion

Clase para los objetos de simulacion y poder utilizarlos como variables globales

In [14]:
#Class for simulation object
class sim_objects ():
    def __init__(self):
        self.client = 0
        self.sim = 0
        self.file = 0
        self.random_positions = []
        self.previous = True

sim_obj = sim_objects()

No se usan los siguientes porque se creo una clase

In [7]:
#Always add at the begining of every code for simulations after the libraries import
client = RemoteAPIClient()
sim = client.getObject('sim')

#### Funciones

##### Handlers del robot

###### No se usan

In [5]:
"""
#Defining handlers
def load_quad():
    #Right-front
    JRF0 = sim.getObject("/JRF0")
    JRF1 = sim.getObject("/JRF1")
    JRF2 = sim.getObject("/JRF2")
    #Right-back
    JRB0 = sim.getObject("/JRB0")
    JRB1 = sim.getObject("/JRB1")
    JRB2 = sim.getObject("/JRB2")
    #Left-front
    JLF0 = sim.getObject("/JLF0")
    JLF1 = sim.getObject("/JLF1")
    JLF2 = sim.getObject("/JLF2")
    #Left-back
    JLB0 = sim.getObject("/JLB0")
    JLB1 = sim.getObject("/JLB1")
    JLB2 = sim.getObject("/JLB2")
    #Head
    Head = sim.getObject("/Head")
    Base = sim.getObject("/Base")
    joint_handlers = [JRF0, JRF1, JRF2, JRB0, JRB1, JRB2, JLF0, JLF1, JLF2, JLB0, JLB1, JLB2]
    shape_handlers = [Base, Head]
    return joint_handlers, shape_handlers
"""

'\n#Defining handlers\ndef load_quad():\n    #Right-front\n    JRF0 = sim.getObject("/JRF0")\n    JRF1 = sim.getObject("/JRF1")\n    JRF2 = sim.getObject("/JRF2")\n    #Right-back\n    JRB0 = sim.getObject("/JRB0")\n    JRB1 = sim.getObject("/JRB1")\n    JRB2 = sim.getObject("/JRB2")\n    #Left-front\n    JLF0 = sim.getObject("/JLF0")\n    JLF1 = sim.getObject("/JLF1")\n    JLF2 = sim.getObject("/JLF2")\n    #Left-back\n    JLB0 = sim.getObject("/JLB0")\n    JLB1 = sim.getObject("/JLB1")\n    JLB2 = sim.getObject("/JLB2")\n    #Head\n    Head = sim.getObject("/Head")\n    Base = sim.getObject("/Base")\n    joint_handlers = [JRF0, JRF1, JRF2, JRB0, JRB1, JRB2, JLF0, JLF1, JLF2, JLB0, JLB1, JLB2]\n    shape_handlers = [Base, Head]\n    return joint_handlers, shape_handlers\n'

In [3]:
#Defining handlers
def load_quad():
    base = sim.getObject("/Base")
    #Right-front
    JRF0 = sim.getObject("/JRF0")
    JRF1 = sim.getObject("/JRF1")
    JRF2 = sim.getObject("/JRF2")
    #Right-back
    JRB0 = sim.getObject("/JRB0")
    JRB1 = sim.getObject("/JRB1")
    JRB2 = sim.getObject("/JRB2")
    #Left-front
    JLF0 = sim.getObject("/JLF0")
    JLF1 = sim.getObject("/JLF1")
    JLF2 = sim.getObject("/JLF2")
    #Left-back
    JLB0 = sim.getObject("/JLB0")
    JLB1 = sim.getObject("/JLB1")
    JLB2 = sim.getObject("/JLB2")

    joint_handlers = [JRF0, JRF1, JRF2, JRB0, JRB1, JRB2, JLF0, JLF1, JLF2, JLB0, JLB1, JLB2]
    return joint_handlers, base

###### Se utilizara
Se utilizaran los handlers dentro de una clase para poder utilizarlos como variables globales

In [7]:
# Class to store the different handlers the robot needs to operate in the simulator. 
class handlers ():
    def __init__(self):
        self.JRF0 = 0
        self.JRF1 = 0
        #self.JRF2 = 0
        self.JRB0 = 0
        self.JRB1 = 0
        #self.JRB2 = 0
        self.JLF0 = 0
        self.JLF1 = 0
        #self.JLF2 = 0
        self.JLB0 = 0
        self.JLB1 = 0
        #self.JLB2 = 0
        self.base = 0

handler = handlers()

Funcion para crear los handlers

In [8]:
# Function that load the robot in the scene at the beggining of each evaluation.
def load_quad_class():
    # Opening the connection with CoppeliaSim simulator
    sim_obj.client = RemoteAPIClient()
    sim_obj.sim = sim_obj.client.getObject('sim')
    
    # Setting the simulation mode for working in sinchronous mode. The simulation timming is controlled by the Python program and not by the simulation itselfs.
    sim_obj.client.setStepping(True)
    
    handler.base = sim.getObject("/Base")
    #Right-front
    handler.JRF0 = sim_obj.sim.getObject("/JRF0")
    handler.JRF1 = sim_obj.sim.getObject("/JRF1")
    handler.JRF2 = sim_obj.sim.getObject("/JRF2")
    #Right-back
    handler.JRB0 = sim_obj.sim.getObject("/JRB0")
    handler.JRB1 = sim_obj.sim.getObject("/JRB1")
    handler.JRB2 = sim_obj.sim.getObject("/JRB2")
    #Left-front
    handler.JLF0 = sim_obj.sim.getObject("/JLF0")
    handler.JLF1 = sim_obj.sim.getObject("/JLF1")
    handler.JLF2 = sim_obj.sim.getObject("/JLF2")
    #Left-back
    handler.JLB0 = sim_obj.sim.getObject("/JLB0")
    handler.JLB1 = sim_obj.sim.getObject("/JLB1")
    handler.JLB2 = sim_obj.sim.getObject("/JLB2")

##### Set de posiciones random

Seran angulos pequeños angulos aleatorios para simular angulos incrementales a partir de la posicion angular anterior

In [12]:
#Generate list of random positions
def rand_gen():
    randomlist = []
    for i in range(9):
        n = rn(-10, 10) * math.pi / 180
        randomlist.append(n)
    return randomlist

##### Toma de datos

* Posicion (angular) de las joints con respecto a su eje de giro.
* Posicion y orientacion absoluta de la base.

In [13]:
"""
def get_joints_preceptions(handlers, base):
    positions = []
    orientations = []
    base_pos = sim.getObjectPosition(base, sim.handle_world)
    base_ori = sim.getObjectOrientation(base, sim.handle_world)
    for i in handlers:
        pos = sim.getObjectPosition(i,sim.handleflag_reljointbaseframe)
        positions.append(pos)
        ori = sim.getJointPosition(i)
        orientations.append(ori)
    return positions, orientations, base_pos, base_ori
"""

'\ndef get_joints_preceptions(handlers, base):\n    positions = []\n    orientations = []\n    base_pos = sim.getObjectPosition(base, sim.handle_world)\n    base_ori = sim.getObjectOrientation(base, sim.handle_world)\n    for i in handlers:\n        pos = sim.getObjectPosition(i,sim.handleflag_reljointbaseframe)\n        positions.append(pos)\n        ori = sim.getJointPosition(i)\n        orientations.append(ori)\n    return positions, orientations, base_pos, base_ori\n'

Obtiene las percepciones que en la realidad vendran de diferentes sensores

In [15]:
def get_joints_preceptions_v1(handlers, base):
    orientations = []
    base_pos = sim.getObjectPosition(base, sim.handle_world)
    base_ori = sim.getObjectOrientation(base, sim.handle_world)
    for i in handlers:
        ori = sim.getJointPosition(i)
        orientations.append(ori)
    return orientations, base_pos, base_ori

Version nueva que incluye el torque y velocidades de las articulaciones

In [ ]:
def get_joints_preceptions(handlers, base):
    positions = []
    forces = []
    velocities = []
    l_velocities = []
    base_pos = sim.getObjectPosition(base, sim.handle_world)
    base_ori = sim.getObjectOrientation(base, sim.handle_world)
    for i in handlers:
        #Angular/linealr position of the joint
        pos = sim.getJointPosition(i)
        positions.append(pos)
        #Torque/force being aplied by the joint
        force = sim.getJointForce(i)
        forces.append(force)
        #Angular/ linear velocity of the joint's movement
        veloc = sim.getJointVelocity(i)
        velocities.append(veloc)
        #Linear velocity of the joint
        l_veloc = sim.getJointVelocity(i)
        l_velocities.append(l_veloc)
    return positions, forces, velocities, l_velocities, base_pos, base_ori

### Ejecucion

Funcion para ejecutar las posiciones random generadas

In [44]:
def main():
    sim.startSimulation()
    while (t := sim.getSimulationTime()) < 2:
        joint_handlers, base = load_quad()
        rand_pos = rand_gen()
        for i in range(0,len(rand_pos)):
            #print (joint_handlers[i], positions[i])
            sim.setJointTargetPosition(joint_handlers[i], rand_pos[i])
            client.step()
        #s = f'Simulation time: {t:.2f} [s]'
        #print(s)
        client.step()
    sim.stopSimulation()

Funcion para obtener datos previos y posteriores, y ejecutar las posiciones random generadas

In [18]:
"""
def main():
    sim.startSimulation()
    joint_handlers, base = load_quad()
    rand_pos = rand_gen()
    prev_pos, prev_ori, prev_base_pos, prev_base_ori = get_joints_preceptions(joint_handlers, base)
    for i in range(0,len(rand_pos)):
        sim.setJointTargetPosition(joint_handlers[i], rand_pos[i])
        client.step()
    post_pos, post_ori, post_base_pos, post_base_ori = get_joints_preceptions(joint_handlers, base)
    print(prev_pos)
    print(post_pos)
    print(prev_ori)
    print(post_ori)
    print(prev_base_pos)
    print(post_base_pos)
    print(prev_base_ori)
    print(post_base_ori)
    sim.stopSimulation()
    """

'\ndef main():\n    sim.startSimulation()\n    joint_handlers, base = load_quad()\n    rand_pos = rand_gen()\n    prev_pos, prev_ori, prev_base_pos, prev_base_ori = get_joints_preceptions(joint_handlers, base)\n    for i in range(0,len(rand_pos)):\n        sim.setJointTargetPosition(joint_handlers[i], rand_pos[i])\n        client.step()\n    post_pos, post_ori, post_base_pos, post_base_ori = get_joints_preceptions(joint_handlers, base)\n    print(prev_pos)\n    print(post_pos)\n    print(prev_ori)\n    print(post_ori)\n    print(prev_base_pos)\n    print(post_base_pos)\n    print(prev_base_ori)\n    print(post_base_ori)\n    sim.stopSimulation()\n    '

Se ejecuta una secuencia de posiciones y se obtienen las diferentes medidas

In [17]:
def main():
    sim.startSimulation()
    joint_handlers, base = load_quad()
    rand_pos = rand_gen()
    prev_ori, prev_base_pos, prev_base_ori = get_joints_preceptions_v1(joint_handlers, base)
    for i in range(0,len(rand_pos)):
        sim.setJointTargetPosition(joint_handlers[i], rand_pos[i])
        client.step()
    post_ori, post_base_pos, post_base_ori = get_joints_preceptions_v1(joint_handlers, base)
    print("Pos angular previa: ", prev_ori)
    print("Pos angular por ejecutar: ", rand_pos)
    print("Pros angular posterior: ", post_ori)
    print("Posicion previa base (x,y,z): ", prev_base_pos)
    print("Posicion posterior base (x,y,z): ",post_base_pos)
    print("Orientacion previa base (alpha, beta, gamma): ",prev_base_ori)
    print("Orientacion posterior base (alpha, beta, gamma): ",post_base_ori)
    sim.pauseSimulation()

In [59]:
main()

Pos angular previa:  [-0.00027751922607421875, 0.00066375732421875, 0.0002608299255371094, 0.0009446144104003906, 0.00044345855712890625, 9.870529174804688e-05, -0.00078582763671875, 0.0003376007080078125, 0.0001506805419921875, -0.0009438991546630859, 0.0005726814270019531, 0.0004949569702148438]
Pos angular por ejecutar:  [-0.3839724354387525, 0.10471975511965977, 1.0821041362364843, 0.3490658503988659, 1.1344640137963142, 1.4660765716752369, 0.15707963267948966, -0.4886921905584123, -0.9773843811168246, 1.3788101090755203, -0.9948376736367678, -1.43116998663535]
Pros angular posterior:  [-0.1879103183746338, 0.14304780960083008, 1.115992784500122, 0.0773930549621582, 0.37836599349975586, 1.50472092628479, 0.1588902473449707, -0.4954569339752197, -1.0048339366912842, 1.4134438037872314, -1.0140836238861084, -1.469933271408081]
Posicion previa base (x,y,z):  [-4.62464186057332e-06, 1.1430309314164333e-05, 0.11770223081111908]
Posicion posterior base (x,y,z):  [0.008931975811719894, -0

In [55]:
sim.stopSimulation()

1

Guardar las mediciones en una sola lista seguida con todas las iteraciones

### Exportar

In [5]:
path = os.getcwd()
print(path)

c:\Users\carlo\OneDrive - Estudiantes ITCR\U\TFG España\Codigo


In [6]:
#sim_obj.file = open(path + r'\training.txt', 'wb')
#pickle.dump(path, sim_obj.file)
#sim_obj.file.close()

In [16]:
#Dictionary tests

class a ():
    def __init__(self):
        #Initial perceptions
        self.prev_j_positions = 1
        self.prev_j_velocities = 2
        self.prev_j_l_velocities = 3
        self.prev_j_forces = 3
        self.prev_base_pos = 6
        self.prev_base_ori = 7
        self.prev_base_accel = 23

ass = a()
ass_dict = ass.__dict__
print(ass_dict)
print(ass.__dict__.values())

{'prev_j_positions': 1, 'prev_j_velocities': 2, 'prev_j_l_velocities': 3, 'prev_j_forces': 3, 'prev_base_pos': 6, 'prev_base_ori': 7, 'prev_base_accel': 23}
dict_values([1, 2, 3, 3, 6, 7, 23])


## Cambios para que funcione con clases

Para exportar se ve que crear una clase para almacenar todas las percepciones es muy util por lo que se procede a modificar todas las funciones anteriores que lo requieran para que funcionen con la clase de percepciones.

#### Librerias

In [35]:
import os
import math
import pickle
from zmqRemoteApi import RemoteAPIClient
from random import randint as rn

#### Creacion de clases

In [33]:
#Class for simulation object
class sim_objects ():
    def __init__(self):
        self.client = 0
        self.sim = 0
        self.file = 0
        self.random_positions = []
        self.executed = False

sim_obj = sim_objects()

# Class to store the different handlers the robot needs to operate in the simulator. 
class handlers_class ():
    def __init__(self):
        self.JRF0 = 0
        self.JRF1 = 0
        #self.JRF2 = 0
        self.JRB0 = 0
        self.JRB1 = 0
        #self.JRB2 = 0
        self.JLF0 = 0
        self.JLF1 = 0
        #self.JLF2 = 0
        self.JLB0 = 0
        self.JLB1 = 0
        #self.JLB2 = 0
        self.base = 0

handler = handlers_class()
handler_values = handler.__dict__.values()

#### Funciones de carga del quad y generacion de acciones random

In [ ]:
# Function that load the robot in the scene at the beggining of each evaluation.
def load_quad_class():
    # Opening the connection with CoppeliaSim simulator
    sim_obj.client = RemoteAPIClient()
    sim_obj.sim = sim_obj.client.getObject('sim')
    
    # Setting the simulation mode for working in sinchronous mode. The simulation timming is controlled by the Python program and not by the simulation itselfs.
    sim_obj.client.setStepping(True)
    
    handler.base = sim_obj.sim.getObject("/Base")
    #Right-front
    handler.JRF0 = sim_obj.sim.getObject("/JRF0")
    handler.JRF1 = sim_obj.sim.getObject("/JRF1")
    #handler.JRF2 = sim_obj.sim.getObject("/JRF2")
    #Right-back
    handler.JRB0 = sim_obj.sim.getObject("/JRB0")
    handler.JRB1 = sim_obj.sim.getObject("/JRB1")
    #handler.JRB2 = sim_obj.sim.getObject("/JRB2")
    #Left-front
    handler.JLF0 = sim_obj.sim.getObject("/JLF0")
    handler.JLF1 = sim_obj.sim.getObject("/JLF1")
    #handler.JLF2 = sim_obj.sim.getObject("/JLF2")
    #Left-back
    handler.JLB0 = sim_obj.sim.getObject("/JLB0")
    handler.JLB1 = sim_obj.sim.getObject("/JLB1")
    #handler.JLB2 = sim_obj.sim.getObject("/JLB2")

In [28]:
#Generate list of random positions
def rand_gen():
    randomlist = []
    for i in range(9):
        n = rn(-10, 10) * math.pi / 180
        randomlist.append(n)
    sim_obj.random_positions.append(randomlist)

#### Clase y funcion para percepciones

In [38]:

#Class to store all the perceptions
class perceptions ():
    def __init__(self):
        #Initial perceptions
        self.prev_j_positions = []
        self.prev_j_velocities = []
        self.prev_j_l_velocities = []
        self.prev_j_forces = []
        self.prev_base_pos = []
        self.prev_base_ori = []
        self.prev_base_accel = []

        #Resulting perceptions
        self.post_j_positions = []
        self.post_j_velocities = []
        self.post_j_l_velocities = []
        self.post_j_forces = []
        self.post_base_pos = []
        self.post_base_ori = []
        self.post_base_accel = []

perception = perceptions()


def get_joints_preceptions():
    positions = []
    forces = []
    velocities = []
    l_velocities = []
    base_pos = sim_obj.sim.getObjectPosition(handler.base, sim_obj.sim.handle_world)
    base_ori = sim_obj.sim.getObjectOrientation(handler.base, sim_obj.sim.handle_world)
    base_accel = [] #sim_obj.sim.getObjectOrientation(handler.base, sim_obj.sim.handle_world) --- MISSING
    for i in handler.__dict__.values():
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(i)
        positions.append(pos)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(i)
        forces.append(force)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(i)
        velocities.append(veloc)
        #Linear velocity of the joint
        l_veloc = sim_obj.sim.getJointVelocity(i)
        l_velocities.append(l_veloc)
    if  not sim_obj.executed:
        perception.prev_j_positions.append(positions)
        perception.prev_j_velocities.append(velocities)
        perception.prev_j_l_velocities.append(l_velocities)
        perception.prev_j_forces.append(forces)
        perception.prev_base_pos.append(base_pos)
        perception.prev_base_ori.append(base_ori)
        perception.prev_base_accel.append(base_accel)
    else:
        perception.post_j_positions.append(positions)
        perception.post_j_velocities.append(velocities)
        perception.post_j_l_velocities.append(l_velocities)
        perception.post_j_forces.append(forces)
        perception.post_base_pos.append(base_pos)
        perception.post_base_ori.append(base_ori)
        perception.post_base_accel.append(base_accel)

#### Pruebas varias

In [34]:
#sim_obj.executed = False

if  not sim_obj.executed:
    print('not executed')
else:
    print('executed')

not executed


In [32]:
#sim_obj.executed = True

if  not sim_obj.executed:
    print('not executed')
else:
    print('executed')

executed


In [24]:
print(f'{len(handler.__dict__.values())} handlers: {handler.__dict__.values()}')

9 handlers: dict_values([0, 0, 0, 0, 0, 0, 0, 0, 0])


In [20]:
#Stores every perception individually
'''
def get_joints_preceptions():
    if sim_obj.previous:
        perception.prev_base_pos.append(sim.getObjectPosition(handler.base, sim.handle_world))
        perception.prev_base_ori.append(sim.getObjectOrientation(handler.base, sim.handle_world))
        #perception.prev_base_accel = is missing
        #perception.prev_base_accel.append(sim.getObjectOrientation(handler.base, sim.handle_world))

        for i in handler.__dict__.values():
            #Angular/linear position of the joint
            perception.prev_j_positions.append(sim.getJointPosition(i))

            #Torque/force being aplied by the joint
            perception.prev_j_forces.append(sim.getJointForce(i))

            #Angular/linear velocity of the joint's movement
            perception.prev_j_velocities.append(sim.getJointVelocity(i))

            #Linear velocity of the joint (missing function)
            perception.prev_j_l_velocities.append(sim.getJointVelocity(i))
    else:
        perception.post_base_pos.append(sim.getObjectPosition(handler.base, sim.handle_world))
        perception.post_base_ori.append(sim.getObjectOrientation(handler.base, sim.handle_world))
        #perception.post_base_accel = is missing
        #perception.post_base_accel.append(sim.getObjectOrientation(handler.base, sim.handle_world))
        
        for i in handler.__dict__.values():
            #Angular/linear position of the joint
            perception.post_j_positions.append(sim.getJointPosition(i))

            #Torque/force being aplied by the joint
            perception.post_j_forces.append(sim.getJointForce(i))

            #Angular/linear velocity of the joint's movement
            perception.post_j_velocities.append(sim.getJointVelocity(i))

            #Linear velocity of the joint (missing function)
            perception.post_j_l_velocities.append(sim.getJointVelocity(i))
'''

"\ndef get_joints_preceptions():\n    if sim_obj.previous:\n        perception.prev_base_pos.append(sim.getObjectPosition(handler.base, sim.handle_world))\n        perception.prev_base_ori.append(sim.getObjectOrientation(handler.base, sim.handle_world))\n        #perception.prev_base_accel = is missing\n        #perception.prev_base_accel.append(sim.getObjectOrientation(handler.base, sim.handle_world))\n\n        for i in handler.__dict__.values():\n            #Angular/linear position of the joint\n            perception.prev_j_positions.append(sim.getJointPosition(i))\n\n            #Torque/force being aplied by the joint\n            perception.prev_j_forces.append(sim.getJointForce(i))\n\n            #Angular/linear velocity of the joint's movement\n            perception.prev_j_velocities.append(sim.getJointVelocity(i))\n\n            #Linear velocity of the joint (missing function)\n            perception.prev_j_l_velocities.append(sim.getJointVelocity(i))\n    else:\n        per

#### Ejecucion

In [37]:
def main():
    sim_obj.sim.startSimulation()
    load_quad_class()
    rand_gen()
    get_joints_preceptions()
    for i in range(len(sim_obj.random_positions)):
        sim_obj.sim.setJointTargetPosition(handler.__dict__.values()[i], sim_obj.random_positions[i])
        sim_obj.client.step()
    sim_obj.executed = True
    get_joints_preceptions()
    sim_obj.executed = False
    sim_obj.sim.pauseSimulation()

In [41]:
#main()

In [42]:
#sim_obj.sim.stopSimulation()

In [39]:
path = os.getcwd()
print(path)

c:\Users\carlo\OneDrive - Estudiantes ITCR\U\TFG España\Codigo


In [46]:
sim_obj.file = open(os.getcwd() + r'\training.txt', 'wb')
pickle.dump(path, sim_obj.file)
sim_obj.file.close()